# Fitness Club Analytics
## Анализ клиентской активности сети фитнес-клубов

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### 1. Загрузка данных

In [ ]:
workouts = pd.read_csv('workouts.csv')
payments = pd.read_excel('payments.xlsx')
users = pd.read_excel('users.xlsx')

workouts['start_at'] = pd.to_datetime(workouts['start_at'])
payments['payment_date'] = pd.to_datetime(payments['payment_date'])

### 2. Количество тренировок по клиентам

In [ ]:
workout_stats = workouts.groupby('client_id').agg(
    workout_count=('workout_id', 'count')
).reset_index()

### 3. Сумма платежей и первая дата

In [ ]:
payment_stats = payments.groupby('user_id').agg(
    total_payments=('amount', 'sum'),
    first_payment=('payment_date', 'min')
).reset_index()

### 4. Сборка общей таблицы

In [ ]:
df = users.merge(
    workout_stats, left_on='user_id', right_on='client_id', how='left'
).merge(
    payment_stats, on='user_id', how='left'
)

df['workout_count'] = df['workout_count'].fillna(0)
df['total_payments'] = df['total_payments'].fillna(0)
df['is_payer'] = df['total_payments'] > 0

df.head()

### 5. Средний возраст платящих и неплатящих

In [ ]:
avg_age_payer = df[df['is_payer'] == True]['age'].mean()
avg_age_nonpayer = df[df['is_payer'] == False]['age'].mean()

print(f'Средний возраст платящих: {avg_age_payer:.1f}')
print(f'Средний возраст неплатящих: {avg_age_nonpayer:.1f}')

### 6. Доля неплатящих

In [ ]:
nonpayer_share = len(df[df['is_payer'] == False]) / len(df)
print(f'Доля клиентов без платежей: {nonpayer_share:.1%}')

### 7. Динамика платежей по месяцам

In [ ]:
payments['month'] = payments['payment_date'].dt.to_period('M')

monthly = payments.groupby('month').agg(
    amount=('amount', 'sum'),
    count=('payment_id', 'count')
).reset_index()

monthly['month_str'] = monthly['month'].astype(str)
monthly.head()

### 8. Визуализация: сумма и количество платежей

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.bar(monthly['month_str'], monthly['amount'], alpha=0.7, color='blue', label='Сумма')
ax1.set_xlabel('Месяц')
ax1.set_ylabel('Сумма платежей', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(monthly['month_str'], monthly['count'], color='red', marker='o', label='Количество')
ax2.set_ylabel('Количество платежей', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Динамика платежей по месяцам')
fig.autofmt_xdate(rotation=45)
plt.tight_layout()
plt.show()

### 9. Средний чек по месяцам

In [ ]:
monthly['avg_check'] = monthly['amount'] / monthly['count']

plt.figure(figsize=(10, 4))
plt.scatter(monthly['month_str'], monthly['avg_check'], color='green', s=80)
plt.plot(monthly['month_str'], monthly['avg_check'], color='green', linestyle='--', alpha=0.5)
plt.title('Средний чек по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Средний чек')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### 10. Краткие выводы

1. Средний возраст **платящих клиентов выше**, чем неплатящих.
2. Доля клиентов **без платежей** требует внимания — возможна работа с реактивацией.
3. Пик платежей приходится на **осенние месяцы**, спад — на начало года.
4. Средний чек **стабилен**, резких скачков нет.